<a href="https://colab.research.google.com/github/larasauser/master/blob/main/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Baseline method

In [16]:
!pip install rasterio

In [17]:
import os
import re
import numpy as np
import rasterio
from rasterio.warp import reproject, Resampling
from datetime import datetime

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path_with_holes = "/content/drive/MyDrive/NDVI_grancy_Landsat8_10/imputed/holes_real"
path_full = "/content/drive/MyDrive/NDVI_grancy_Landsat8_10/imputed"
path_out = "/content/drive/MyDrive/NDVI_grancy_Landsat8_10/imputed/holes_real/filled_baseline"
os.makedirs(path_out, exist_ok=True)

In [ ]:
def extract_date(filename):
    match = re.search(r"(\d{4}-\d{2}-\d{2})", filename)
    if match:
        return datetime.strptime(match.group(1), "%Y-%m-%d")
    else:
        raise ValueError(f"Impossible to extract : {filename}")

In [ ]:
files_with_holes = sorted([f for f in os.listdir(path_with_holes) if f.endswith(".tif")])
files_full = sorted([f for f in os.listdir(path_full) if f.endswith(".tif")])
dates_full = {f: extract_date(f) for f in files_full}

In [ ]:
def find_closest_image(target_date, exclude_date):
    candidates = [(f, d) for f, d in dates_full.items() if d != exclude_date]
    closest = min(candidates, key=lambda x: abs(x[1] - target_date))
    return closest[0]

In [ ]:
# Reconstruction
for f in files_with_holes:
    date_f = extract_date(f)

    # Find closest image in time
    closest_file = find_closest_image(date_f, date_f)

    # Load image with hole
    with rasterio.open(os.path.join(path_with_holes, f)) as src_hole:
        img_hole = src_hole.read(1).astype("float32")
        profile = src_hole.profile
        target_transform = src_hole.transform
        target_crs = src_hole.crs
        target_shape = src_hole.shape

    # Load image full
    with rasterio.open(os.path.join(path_full, closest_file)) as src_full:
        cand_img = src_full.read(1).astype("float32")
        cand_transform = src_full.transform
        cand_crs = src_full.crs

    # Reprojection + recadrage pour matcher l'image trouée
    recropped = np.empty(target_shape, dtype="float32")
    reproject(
        source=cand_img,
        destination=recropped,
        src_transform=cand_transform,
        src_crs=cand_crs,
        dst_transform=target_transform,
        dst_crs=target_crs,
        resampling=Resampling.bilinear
    )

    # Construire masque des trous : NaN ou -9999
    mask = np.isnan(img_hole) | (img_hole == -9999)

    # Remplir les trous avec l'image reprojetée
    img_out = img_hole.copy()
    img_out[mask] = recropped[mask]

    # Préparer profil de sortie
    profile.update(dtype="float32", nodata=-9999)

    # Remplacer NaN par -9999 pour cohérence
    img_out[np.isnan(img_out)] = -9999

    # Sauvegarde
    out_path = os.path.join(path_out, f.replace(".tif", "_baseline.tif"))
    with rasterio.open(out_path, "w", **profile) as dst:
        dst.write(img_out, 1)

    print(f"{f} rempli avec {closest_file} → {out_path}")

Visualisation + error calculation